# SQL Subqueries - Lab

## Introduction

Now that you've seen how subqueries work, it's time to get some practice writing them! Not all of the queries will require subqueries, but all will be a bit more complex and require some thought and review about aggregates, grouping, ordering, filtering, joins and subqueries. Good luck!  

## Objectives

You will be able to:

* Write subqueries to decompose complex queries

## CRM Database ERD

Once again, here's the schema for the CRM database you'll continue to practice with.

<img src="images/Database-Schema.png" width="600">

## Connect to the Database

As usual, start by importing the necessary packages and connecting to the database `data.sqlite`.

In [1]:
import pandas as pd
import sqlite3
conn = sqlite3.connect('data.sqlite')

## Write an Equivalent Query using a Subquery

The following query works using a `JOIN`. Rewrite it so that it uses a subquery instead.

```
SELECT
    customerNumber,
    contactLastName,
    contactFirstName
FROM customers
JOIN orders 
    USING(customerNumber)
WHERE orderDate = '2003-01-31'
;
```

In [3]:
pd.read_sql("""
    SELECT
        customerNumber,
        contactLastName,
        contactFirstName
    FROM customers
    WHERE customerNumber in (SELECT customerNumber
                             FROM orders
                             WHERE orderDate = '2003-01-31');
""",conn)

,customerNumber,contactLastName,contactFirstName
0,141,Freyre,Diego


## Select the Total Number of Orders for Each Product Name

Sort the results by the total number of items sold for that product.

In [32]:
pd.read_sql("""
    SELECT 
        productName,
        total_orders
    FROM (
        SELECT
            productCode,
            COUNT(orderNumber) AS total_orders
        FROM orderdetails
        GROUP BY productCode)
    LEFT JOIN 
        products
        USING(productCode)
    ORDER BY total_orders;
""",conn)

,productName,total_orders
0,1957 Ford Thunderbird,24
1,1952 Citroen-15CV,24
2,1965 Aston Martin DB5,25
3,1917 Grand Touring Sedan,25
4,1911 Ford Town Car,25
...,...,...
104,ATA: B757-300,28
105,F/A 18 Hornet 1/72,28
106,American Airlines: MD-11S,28
107,Boeing X-32A JSF,28


## Select the Product Name and the  Total Number of People Who Have Ordered Each Product

Sort the results in descending order.

### A quick note on the SQL  `SELECT DISTINCT` statement:

The `SELECT DISTINCT` statement is used to return only distinct values in the specified column. In other words, it removes the duplicate values in the column from the result set.

Inside a table, a column often contains many duplicate values; and sometimes you only want to list the unique values. If you apply the `DISTINCT` clause to a column that has `NULL`, the `DISTINCT` clause will keep only one NULL and eliminates the other. In other words, the DISTINCT clause treats all `NULL` “values” as the same value.

In [17]:
pd.read_sql("""
    SELECT productName,
           COUNT(DISTINCT(customerNumber)) AS num_cust
    FROM products
    JOIN orderDetails AS od
        USING(productCode)
    JOIN orders as o
        USING(orderNumber)
    GROUP BY productCode
    ORDER BY num_cust DESC;
""",conn)

,productName,num_cust
0,1992 Ferrari 360 Spider red,40
1,Boeing X-32A JSF,27
2,1934 Ford V8 Coupe,27
3,1972 Alfa Romeo GTA,27
4,1952 Alpine Renault 1300,27
...,...,...
104,1958 Chevy Corvette Limited Edition,19
105,2002 Chevy Corvette,18
106,1969 Chevrolet Camaro Z28,18
107,1952 Citroen-15CV,18


## Select the Employee Number, First Name, Last Name, City (of the office), and Office Code of the Employees Who Sold Products That Have Been Ordered by Fewer Than 20 people.

This problem is a bit tougher. To start, think about how you might break the problem up. Be sure that your results only list each employee once.

In [30]:
# Written all out. 
pd.read_sql("""
    SELECT employeeNumber,
           firstName,
           lastName,
           city,
           officeCode
    FROM employees
    JOIN offices
        USING(officeCode)
    WHERE employeeNumber in (
        SELECT DISTINCT employeeNumber
        FROM employees AS e
        JOIN customers AS c
            ON e.employeeNumber = c.salesRepEmployeeNumber
        JOIN orders
            USING(customerNumber)
        JOIN orderdetails AS od
            USING(orderNumber)
        WHERE productCode in (
            SELECT productCode
            FROM products
            JOIN orderDetails AS od
                USING(productCode)
            JOIN orders as o
                USING(orderNumber)
            GROUP BY productCode
                HAVING COUNT(DISTINCT(customerNumber)) < 20));
""",conn)

,employeeNumber,firstName,lastName,city,officeCode
0,1165,Leslie,Jennings,San Francisco,1
1,1166,Leslie,Thompson,San Francisco,1
2,1188,Julie,Firrelli,Boston,2
3,1216,Steve,Patterson,Boston,2
4,1286,Foon Yue,Tseng,NYC,3
5,1323,George,Vanauf,NYC,3
6,1337,Loui,Bondur,Paris,4
7,1370,Gerard,Hernandez,Paris,4
8,1401,Pamela,Castillo,Paris,4
9,1501,Larry,Bott,London,7


In [26]:
# subquery to find all of the product codes which have been ordered by fewer than 20 people
pd.read_sql("""
    SELECT productCode
    FROM products
    JOIN orderDetails AS od
        USING(productCode)
    JOIN orders as o
        USING(orderNumber)
    GROUP BY productCode
    HAVING COUNT(DISTINCT(customerNumber)) < 20;
""",conn)

,productCode
0,S24_2766
1,S24_2840
2,S24_2887
3,S24_3191
4,S24_3432


In [22]:
# subquery to find all of the products that each employee has sold 
pd.read_sql("""
    SELECT e.employeeNumber,
           od.productCode
    FROM employees AS e
    JOIN customers AS c
        ON e.employeeNumber = c.salesRepEmployeeNumber
    JOIN orders
        USING(customerNumber)
    JOIN orderdetails AS od
        USING(orderNumber);
""",conn)

,employeeNumber,productCode
0,1165,S12_1666
1,1165,S18_1097
2,1165,S18_4668
3,1165,S32_3522
4,1165,S12_1099
...,...,...
2991,1702,S32_2509
2992,1702,S32_3207
2993,1702,S50_1392
2994,1702,S18_2248


In [29]:
# combining the two queries to output employee numbers of any employees 
# who have sold products which have been ordered by fewer than 20 people. 
pd.read_sql("""
    SELECT DISTINCT employeeNumber
    FROM employees AS e
    JOIN customers AS c
        ON e.employeeNumber = c.salesRepEmployeeNumber
    JOIN orders
        USING(customerNumber)
    JOIN orderdetails AS od
        USING(orderNumber)
    WHERE productCode in (SELECT productCode
                          FROM products
                          JOIN orderDetails AS od
                              USING(productCode)
                          JOIN orders as o
                              USING(orderNumber)
                          GROUP BY productCode
                          HAVING COUNT(DISTINCT(customerNumber)) < 20);
""",conn)

,employeeNumber
0,1370
1,1501
2,1337
3,1166
4,1286
5,1612
6,1611
7,1401
8,1621
9,1323


## Select the Employee Number, First Name, Last Name, and Number of Customers for Employees Whose Customers Have an Average Credit Limit Over 15K

In [45]:
pd.read_sql("""
    SELECT employeeNumber,
           firstName,
           lastName,
           num_cust
    FROM employees AS e
    JOIN (
        SELECT salesRepEmployeeNumber,
        COUNT(DISTINCT customerNumber) AS num_cust
        FROM customers
        GROUP BY salesRepEmployeeNumber) AS n
        ON n.salesRepEmployeeNumber = e.employeeNumber
    WHERE employeeNumber in (
        SELECT salesRepEmployeeNumber     
        FROM customers
        GROUP BY salesRepEmployeeNumber
        HAVING AVG(creditLimit) > 15000)
    ORDER BY num_cust DESC;
""",conn)

,employeeNumber,firstName,lastName,num_cust
0,1401,Pamela,Castillo,10
1,1504,Barry,Jones,9
2,1323,George,Vanauf,8
3,1501,Larry,Bott,8
4,1286,Foon Yue,Tseng,7
5,1370,Gerard,Hernandez,7
6,1165,Leslie,Jennings,6
7,1166,Leslie,Thompson,6
8,1188,Julie,Firrelli,6
9,1216,Steve,Patterson,6


In [41]:
# Number of customers per employee number
pd.read_sql("""
    SELECT salesRepEmployeeNumber as employeeNumber,
           COUNT(DISTINCT customerNumber) AS num_cust
    FROM customers
    GROUP BY employeeNumber
    ORDER BY num_cust DESC;
""",conn)

,employeeNumber,num_cust
0,,22
1,1401,10
2,1504,9
3,1501,8
4,1323,8
5,1370,7
6,1286,7
7,1702,6
8,1337,6
9,1216,6


In [37]:
# Employees whose customers have an average credit limit over 15,000
pd.read_sql("""
    SELECT salesRepEmployeeNumber     
    FROM customers
    GROUP BY salesRepEmployeeNumber
    HAVING AVG(creditLimit) > 15000;
""",conn)

,salesRepEmployeeNumber
0,1165
1,1166
2,1188
3,1216
4,1286
5,1323
6,1337
7,1370
8,1401
9,1501


## Summary

In this lesson, you got to practice some more complex SQL queries, some of which required subqueries. There's still plenty more SQL to be had though; hope you've been enjoying some of these puzzles!